# Grass wave impact example

This tutorial shows how to setup, run and analyse a wave impact calculation with DiKErnel using the dikerosion python toolbox. It deals with:
1. [Specifying schematization](#Dike-profile-schematization)
1. [Hydrodynamic conditions](#hydrodynamic-conditions)
1. [Output location](#Define-calculation-output) for wave impact calculations on a grass revetment
1. [Performing a calculation](#Start-the-calculation)
1. [Analysing wave impact calculation results](#Analyse-results)

<div class="alert alert-block alert-info">
<b>Not necessary:</b> The following code makes sure that also during development the toolbox can be adressed properly.
</div>

In [1]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

## Dike profile schematization
First we need to define the dike geometry and roughnesses. In this simplified example we consider a dike with a slope of 1:3, reaching up to 10 meter above the reference level. The slope is considered to be very smooth (no roughness)

In [2]:
from pydrever.data import DikeSchematization

x_positions = [0.0, 30.0]
z_positions = [0.0, 10.0]
roughnesses = [1]
schematization = DikeSchematization(
    dike_orientation=0.0, 
    x_positions=x_positions, 
    z_positions=z_positions, 
    roughnesses=roughnesses, 
    x_outer_toe=0.0, 
    x_outer_crest=30.0
)

## Hydrodynamic conditions
In this example we consider a forcing with constant hydrodynamic conditions for 1 hour (3600 seconds) with a water level half way the slope (at 5.0 above the reference level) and waves of $H_s$=1.25m and $T_p$=5.3s. The wave direction is slightly (15 degrees) oblique.

In [3]:
from pydrever.data import HydrodynamicConditions

hydrodynamic_conditions = HydrodynamicConditions(
    time_steps=[0, 3600], 
    water_levels=[5.0], 
    wave_heights=[1.25], 
    wave_periods=[5.3], 
    wave_directions=[15.0]
)

## Define calculation output
Now it is time to define which calculation should be performed. In this example we use a 'OutputLocationSpecification' to specify the desired calculation. The location is at the x-position of 15.0 meters cross-shore (so half way the slope at exactly the still water front). The top layer consists out of grass with a closed sod.

In [4]:
from pydrever.data import (
    OutputLocationSpecification, 
    GrassWaveImpactLayerSpecification, 
    TopLayerType)

output_location = OutputLocationSpecification(
    top_layer_specification= GrassWaveImpactLayerSpecification(
        top_layer_type=TopLayerType.GrassClosedSod
        ),
    x_position=15.0
)

## Start the calculation
Next step is to construct the calculation input and start the calculation.

In [5]:
from pydrever.data import DikernelInput
from pydrever.calculation import Dikernel

input = DikernelInput(
    dike_schematization=schematization,
    hydrodynamic_input= hydrodynamic_conditions,
    output_locations=[output_location]
)

calculation = Dikernel(input=input)
result = calculation.run()

## Analyse results
First result of the run method is a bool that indicates whether the calculation was finished succesfully. In case it was not, the warning and error messages mostly give the information why the calculation did not finish.

In [6]:
if not result:
    print ("The run was unsuccesful:")
    for m in calculation.errors:
        print("  %s" % (m,))

In case of a succesful calculation, the output property of the Dikernel object contains a list with output (locations) for each of the specified calculation locations. In this case it contains only one result for the location at a cross-shore distance of 15.0 meter.

All output locations have general information stating whether the revetment failed as a consequence of the hydrodynamic forcing. In case it failed, the locations also present the 'time_of_failure' additional to the calculated 'final_damage'. Also the development of the damage (and damage increment for each time step) can be found.

In [7]:
if calculation.output[0].failed:
    print("  The revetment failed at %d [s]" % (calculation.output[0].time_of_failure,))
else:
    print("  Just some damage, the revetment did not fail.")
print("  Final damage number: %0.2f" % (calculation.output[0].final_damage,))

  Just some damage, the revetment did not fail.
  Final damage number: 1.21


Other properties are more specific to the type of calculation that was performed (wave impact on a grass revetment in this case). The code below gives an overview of the results of the calculation:

In [8]:
props = vars(calculation.output[0])
print(''.join("%s: %s\n" % item for item in props.items()))

x_position: 15.0
time_of_failure: None
damage_development: [1.2079255643374318]
damage_increment: [1.2079255643374318]
z_position: 5.0
minimum_wave_height: 0.2500000000000006
maximum_wave_height: 1.2499650014124648
loading_revetment: [1.0]
upper_limit_loading: [5.0]
lower_limit_loading: [4.375]
wave_angle: [15.0]
wave_angle_impact: [0.9771528853077257]
wave_height_impact: [1.221441106634657]

